<h1 align="center">LING530F: Deep Learning for NLP</h1>
<h1 align="center">Assignment 2 : Implicit Emotion Detection</h1>
<h1 align="center">Long Short Term Memory Network(LSTM) Model</h1>
<h2 align="center"> Arun Rajendran(86611860)</h2>

In [1]:
%%capture
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install GoogleDriveDownloader
import nltk
nltk.download('popular')
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install torchtext
!pip install spacy && python -m spacy download en

In [2]:
# Download the data

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='11surQQr3jbmzHDKgEA5c46CKA23Nf7PA',                                 # Id of file to be downloaded
                                    dest_path='data/dev.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1TokPKns1uZRAW-GDN0fUz259haU2ZZAW',                                 # Id of file to be downloaded
                                    dest_path='data/test.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1oBiUaUHLrXehWF570xLyQNTjD9QU4fPY',                                 # Id of file to be downloaded
                                    dest_path='data/train.csv')                                                    # Destination path

gdd.download_file_from_google_drive(file_id='1ewgcXWBioeMVJlKsZK9gLv3hWxGDqmiz',                                 # Id of file to be downloaded
                                    dest_path='data/trial-v3.labels')                                                    # Destination path

In [3]:
# Replace the fake labels in dev file with correct labels

labels = open('data/trial-v3.labels','r').readlines()
lines = open('data/dev.csv','r').readlines()
correct_lines=[]
for label, line in zip(labels, lines):
        line=line.rstrip('\n')
        text = line.split(',')
        text[0]=label
        correct_line=[''.join(text[0]),''.join(text[1:])]
        correct_lines.append(correct_line)

In [4]:
# Write the clean dev data in new file
out = open('data/clean_dev.csv',mode='w')                                      
for line in correct_lines:
    line[0]=line[0].rstrip('\n')
    out.write(line[0])
    out.write(',')
    out.write(line[1])
    out.write('\n')
out.close()     

In [5]:
# Read train and dev data
%%capture  # As training data has some error lines
import pandas as pd
X_train = pd.read_table('data/train.csv',sep=',',index_col=False, error_bad_lines=False)
X_test = pd.read_csv('data/clean_dev.csv',sep=',')

In [6]:
# Pandas read skips the error lines, then we save the new dataset into separate file

X_train.to_csv('data/clean_train.csv',sep=',',index=False)

In [7]:
import torch 
import torch.nn as nn
import torchvision.datasets as datasets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from collections import Counter,OrderedDict
from keras.utils.vis_utils import *
import random
import math
import nltk

Using TensorFlow backend.


In [8]:
# Define the vector of glove embeddings

%%capture  # Not display output
from torchtext import vocab
# specify the path to the localy saved vectors
vec = vocab.GloVe(name='840B', dim=300)
# vec = vocab.GloVe(name='6B', dim=200)

In [9]:
import torchtext
import spacy
import re
from torchtext import data

# tokenizer function using spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

def tweet_clean(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove links
    return text.strip()

def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

# define the columns that we want to process and how to process
TEXT = data.Field(sequential=True, 
                 tokenize=tokenizer, 
                 include_lengths=True, 
                 use_vocab=True)
LABEL = data.Field(sequential=False, 
                   use_vocab=True,
                   pad_token=None, 
                   unk_token=None)

train_val_fields = [
    ('label', LABEL), # process it as label
    ('tweet', TEXT) # process it as text
]

trainds, valds = data.TabularDataset.splits(path='data', 
                                            format='csv', 
                                            train='clean_train.csv', 
                                            validation='clean_dev.csv', 
                                            fields=train_val_fields, 
                                            skip_header=True)

In [10]:
# build the vocabulary using train and validation dataset and assign the vectors
TEXT.build_vocab(trainds,valds, max_size=100000, vectors=vec)
# build vocab for labels
LABEL.build_vocab(trainds)

In [11]:
print(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.6339,  0.1782, -0.5805,  ...,  0.3635, -0.3022, -0.0209],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8860, -0.0697, -0.3037,  ...,  0.4366, -0.3720, -0.0778]])


In [12]:
print(LABEL.vocab.vectors)

None


In [13]:
TEXT.vocab.vectors[TEXT.vocab.stoi['under']]

tensor([ 0.1463, -0.2839,  0.1000,  0.2137, -0.1708,  0.0587, -0.1666, -0.2774,
         0.4632,  2.3923, -0.5996,  0.0891, -0.4056,  0.1393, -0.2873, -0.0107,
         0.0960,  1.3774, -0.2206,  0.0566,  0.2584, -0.1883, -0.4987, -0.0209,
        -0.2635, -0.3824, -0.2033,  0.2165,  0.2553, -0.1092, -0.2142, -0.3225,
        -0.2130,  0.1027, -0.1276, -0.5502, -0.3505,  0.2561, -0.1692, -0.1331,
         0.0854,  0.4556,  0.2311, -0.0112,  0.1388, -0.1788, -0.3458, -0.0037,
        -0.1015,  0.4884,  0.1791,  0.1711, -0.6237,  0.2748, -0.1450,  0.1254,
         0.0029, -0.1039,  0.2158, -0.0774,  0.1305, -0.1567,  0.2946,  0.0614,
        -0.2103, -0.0955, -0.3814,  0.2681, -0.3804, -0.0691, -0.4152, -0.3095,
         0.3485,  0.2826,  0.4328, -0.0420,  0.2950, -0.1115,  0.0288, -0.0221,
         0.0126,  0.4712, -0.3743, -0.3939, -0.0773, -0.0103,  0.1367, -0.3248,
         0.4270,  0.4282, -0.0162,  0.0205,  0.4585, -0.2619,  0.3890, -0.1342,
         0.2487,  0.1092,  0.2860, -0.40

In [14]:
# Create bucketiterator to get batches

traindl, valdl = data.BucketIterator.splits(datasets=(trainds, valds), # specify train and validation Tabulardataset
                                            batch_sizes=(64,64),  # batch size of train and validation
                                            sort_key=lambda x: len(x.tweet), # on what attribute the text should be sorted
                                            device=None, # -1 mean cpu and 0 or None mean gpu
                                            sort_within_batch=True, 
                                            repeat=False)

In [15]:
print(len(traindl), len(valdl))

(2365, 150)


In [16]:
traindl

In [17]:
batch = next(iter(traindl)) # BucketIterator return a batch object

In [18]:
print(batch.label) # labels of the batch

tensor([5, 4, 4, 3, 4, 1, 3, 4, 0, 1, 3, 5, 2, 3, 5, 5, 4, 4, 0, 1, 3, 0, 0, 1,
        2, 3, 4, 0, 5, 5, 3, 0, 4, 1, 4, 5, 0, 0, 1, 0, 5, 5, 5, 0, 5, 4, 0, 5,
        5, 3, 5, 0, 5, 5, 0, 3, 2, 2, 2, 2, 3, 3, 2, 5])


In [19]:
print(batch.tweet) 

(tensor([[   3,    3,    5,  ...,   13,    5,  651],
        [  11,   48,    3,  ...,   82,    3,   22],
        [   2,  373,   39,  ..., 7972,   11, 1218],
        ...,
        [  29,   29,   12,  ...,  248,   63,   29],
        [  28,   28, 1737,  ..., 1877,    4,   28],
        [  27,   27,   90,  ..., 4618,  347,   27]]), tensor([26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26]))


In [20]:
print(batch.dataset.fields)

{'tweet': <torchtext.data.field.Field object at 0x7f02e3b31210>, 'label': <torchtext.data.field.Field object at 0x7f02a1ff4390>}


In [21]:
# Create batchgenerator class to use Bucketiterator and attributes to get the batches

class BatchGenerator:
    def __init__(self, dl, x_field, y_field):
        self.dl, self.x_field, self.y_field = dl, x_field, y_field
        
    def __iter__(self):
        for batch in self.dl:
            X = getattr(batch, self.x_field)
            y = getattr(batch, self.y_field)
            yield (X,y)
 


In [22]:
train_batch_it = BatchGenerator(traindl, 'tweet', 'label')
valid_batch_it = BatchGenerator(valdl, 'tweet', 'label')

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
 
# Define the LSTm class        

class SimpleLSTMBaseline(nn.Module):
    def __init__(self, hidden_dim, n_lstm_layers,emb_dim, dropout_p):
        super(SimpleLSTMBaseline,self).__init__() # don't forget to call this!
        self.embedding = nn.Embedding(len(TEXT.vocab), emb_dim)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=False)
        self.encoder = nn.LSTM(emb_dim, hidden_dim, num_layers=n_lstm_layers)
        self.drop = nn.Dropout(dropout_p)
        self.fc1=nn.Linear(hidden_dim,128)
        self.predictor=nn.Linear(128,6)
 
    def forward(self, seq):
        hdn, _ = self.encoder(self.embedding(seq))
        feature = hdn[-1, :, :]
        feature=self.drop(feature)
        feature = F.relu(self.fc1(feature))
        feature=self.drop(feature)
        preds = self.predictor(feature)
        return preds

In [24]:
embed_size = 300
nh = 256
drop=0.5
n_lstm_layers=1
model = SimpleLSTMBaseline(nh,n_lstm_layers,embed_size,drop)

In [25]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.CrossEntropyLoss()
rp = ReduceLROnPlateau(opt,'min',0.1,patience=1, cooldown=0)  
epochs = 5
batch_size = 64 
  
for epoch in range(1, epochs + 1):
    running_loss = 0.0
    running_corrects = 0
    iterations=0
    model.train() # turn on training mode
    for x,y in tqdm(train_batch_it,disable=True): # thanks to our wrapper, we can intuitively iterate over our data!
        iterations=iterations+1
        opt.zero_grad()
        (x,lengths)=x
        predicted = model(x)
        y=y.view(-1)
        loss = loss_func(predicted,y)
        loss.backward()
        opt.step()
        running_loss += loss.item() * x.size(0)
    epoch_loss = running_loss / len(trainds)
 
    # calculate the validation loss for this epoch
    val_loss = 0.0
    list_scores=[]
    model.eval() # turn on evaluation mode
    for x,y in tqdm(valid_batch_it,disable=True):
        (x,lengths)=x
        preds = model(x)
        y=y.view(-1)
        loss = loss_func(preds,y)
        val_loss += loss.item() * x.size(0)
        _,y_pred=torch.max(preds,1)
        temp=f1_score(y_pred,y, average='macro', labels=np.unique(y_pred))
        list_scores.append(temp)
    
    val_loss /= len(valds)
    fscore = sum(list_scores)/(len(valds)/batch_size)
    rp.step(val_loss)
    torch.save(model.state_dict(), 'mytraining'+str(epoch)+'.pt')
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, F1 Sum:{:.4f},Average F1 score(Validation Set): {:.4f}'.format(epoch, epoch_loss, val_loss,sum(list_scores),fscore))


Epoch: 1, Training Loss: 0.2665, Validation Loss: 0.3340, F1 Sum:90.5771,Average F1 score(Validation Set): 0.6079
Epoch: 2, Training Loss: 0.2482, Validation Loss: 0.3393, F1 Sum:90.7791,Average F1 score(Validation Set): 0.6093
Epoch: 3, Training Loss: 0.2303, Validation Loss: 0.3487, F1 Sum:91.3270,Average F1 score(Validation Set): 0.6129
Epoch: 4, Training Loss: 0.1859, Validation Loss: 0.3655, F1 Sum:92.1228,Average F1 score(Validation Set): 0.6183
Epoch: 5, Training Loss: 0.1734, Validation Loss: 0.3786, F1 Sum:91.8780,Average F1 score(Validation Set): 0.6166


In [26]:
from google.colab import files
files.download('mytraining4.pt')

### References

NLTK: Stemming and Lemmatization<br\>
https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization

Apply Stemmer to a column<br\>
https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word

Ensemble: Voting Classifier<br\>
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

Sample Pipeline<br\>
http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

TorchText and LSTM<br\>
http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

TorchText and CNN<br\>
https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/08.CNN-for-Text-Classification.ipynb
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

Saving and loading Pytorch Model<br\>
https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

Self Attention Explained<br\>
https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#self-attention

Self Attention Code for LSTM<br\>
https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py

LSTM + CNN model 
https://discuss.pytorch.org/t/cnn-layer-in-the-top-of-lstm/7941/5
https://discuss.pytorch.org/t/solved-concatenate-time-distributed-cnn-with-lstm/15435/4